<h1>E-periodica data transformation with Spark and ingest to Elasticsearch<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Prerequisites" data-toc-modified-id="Prerequisites-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Prerequisites</a></span></li><li><span><a href="#Processing-e-periodica-data" data-toc-modified-id="Processing-e-periodica-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Processing e-periodica data</a></span><ul class="toc-item"><li><span><a href="#Load-from-AWS-S3" data-toc-modified-id="Load-from-AWS-S3-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Load from AWS S3</a></span></li><li><span><a href="#Get-to-know-the-data" data-toc-modified-id="Get-to-know-the-data-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Get to know the data</a></span></li><li><span><a href="#Cleaning-and-formatting-the-data" data-toc-modified-id="Cleaning-and-formatting-the-data-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Cleaning and formatting the data</a></span></li><li><span><a href="#Text-processing" data-toc-modified-id="Text-processing-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Text processing</a></span><ul class="toc-item"><li><span><a href="#Detecting-languages" data-toc-modified-id="Detecting-languages-2.4.1"><span class="toc-item-num">2.4.1&nbsp;&nbsp;</span>Detecting languages</a></span></li><li><span><a href="#Named-entity-recognition" data-toc-modified-id="Named-entity-recognition-2.4.2"><span class="toc-item-num">2.4.2&nbsp;&nbsp;</span>Named entity recognition</a></span></li></ul></li></ul></li><li><span><a href="#Ingest-to-Elasticsearch" data-toc-modified-id="Ingest-to-Elasticsearch-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Ingest to Elasticsearch</a></span></li><li><span><a href="#Query-Elasticsearch" data-toc-modified-id="Query-Elasticsearch-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Query Elasticsearch</a></span></li></ul></div>

# Prerequisites

In [1]:
# Import basic libraries
import os, re, json
import pandas as pd
print('Successfully imported necessary libraries.')

Successfully imported necessary libraries.


In [2]:
# for I/O AWS S3
!pip install boto3
import boto3

     |████████████████████████████████| 131 kB 30.3 MB/s eta 0:00:01
     |████████████████████████████████| 8.0 MB 67.6 MB/s eta 0:00:01
     |████████████████████████████████| 79 kB 13.1 MB/s eta 0:00:01


In [3]:
# Import the required Python dependencies

import findspark
findspark.init()

# modules for handling dataframes
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, split, lit, when, concat
from pyspark.sql.types import StructType, StructField, ArrayType, DoubleType, IntegerType, StringType

# for user defined functions
from pyspark.sql.functions import udf

In [4]:
# Start Spark Session

import pyspark
conf = pyspark.SparkConf()

# Session configuration
#conf.setMaster("spark://spark-master:7077")
conf.setMaster("local[2]")

conf.set("spark.executor.memory", "8g")
conf.set("spark.executor.cores", "4")
conf.set("spark.driver.memory", "2g")
conf.set("spark.core.connection.ack.wait.timeout", "1200")

# Elasticsearch
conf.set("spark.executor.extraClassPath", "elasticsearch-hadoop-7.12.0/dist/elasticsearch-hadoop-20_2.11-7.12.0.jar, \
            elasticsearch-hadoop-7.12.0/dist/elasticsearch-spark-20_2.11-7.12.0.jar")
conf.set("spark.jars.packages", "org.elasticsearch:elasticsearch-spark-30_2.12:7.12.0")
conf.set("es.index.auto.create", "true")

# Initialize a Spark session with configuration
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('elastic_erara') \
        .config(conf=conf) \
        .getOrCreate()

# Instantiate a Spark Context
sc = spark.sparkContext


In [5]:
spark

In [302]:
#  stop Spark Session, if necessary
spark.stop()

In [6]:
# check executable binaries for the Python interpreter
import sys
print("Python on driver: " + sys.executable)
distData = sc.parallelize(range(100))
python_distros = distData.map(
    lambda x: sys.executable).distinct().collect()
print("Python on nodes: ", python_distros)

Python on driver: /opt/conda/bin/python
Python on nodes:  ['/opt/conda/bin/python']


# Processing e-periodica data

## Load from AWS S3

In [7]:
# Get number of, and keys resp. filepaths of objects in AWS S3

s3 = boto3.client('s3')
response = s3.list_objects(Bucket='bgd-content', MaxKeys=1000, \
                           Prefix='eperiodica-json-fulltext/')    #bernensia-json-fulltext/
#response['ResponseMetadata']
keys = []
file_paths = []
for i in response['Contents'][1:]:
    keys.append(i['Key'])
    file_paths.append('s3a://bgd-content/' + i['Key'])
print(len(keys))
keys[0:5]

916


['eperiodica-json-fulltext/zgh-001:1939:1::293.json',
 'eperiodica-json-fulltext/zgh-001:1939:1::294.json',
 'eperiodica-json-fulltext/zgh-001:1939:1::295.json',
 'eperiodica-json-fulltext/zgh-001:1939:1::296.json',
 'eperiodica-json-fulltext/zgh-001:1939:1::297.json']

In [8]:
def merge_json(path, bigfile):
    '''
    Binds the JSON files of a directy together in one big JSON file.
    
    :param path: path to directory with the JSON files which shall be integrated
    :param bigfile: name of the resulting big JSON file
    '''
    result = list()
    files = [file for file in os.listdir(path) if file.endswith('.json')]
    for f in files:
        with open(path + f, 'r') as infile:
            result.append(json.load(infile))
    with open(bigfile, 'w') as output_file:
        json.dump(result, output_file)

In [ ]:
# AWS S3 example URI 's3a://bgd-content/bernensia-json-fulltext/15251979.json'
# AWS S3 example URI 's3a://bgd-content/eperiodica-json-fulltext/zgh-001:1941:3::313.json'

# make local directory for downloaded JSON files
os.makedirs('download/eperiodica-json-fulltext/')

for i in range(len(keys)):
    s3.download_file('bgd-content', keys[i], 'download/{}'.format(keys[i]))
merge_json('download/eperiodica-json-fulltext/', 'eperiodica_big.json')

# load big JSON file into Spark DataFrame
df = spark.read.json('eperiodica_big.json',  multiLine=True)
df.printSchema()

## Get to know the data

In [11]:
df.show(1, vertical=True)  # show the first row

-RECORD 0---------------------------
 contributor | null                 
 coverage    | null                 
 creator     | ["Boser, Lukas","... 
 date        | 2014                 
 description | null                 
 format      | [text/html, appli... 
 fulltext    | Zur Einleitung
Er... 
 id_intern   | zgh-002:2014:76::406 
 identifier  | [https://www.e-pe... 
 language    | null                 
 publisher   | Rub Media AG         
 relation    | null                 
 rights      | null                 
 source      | [Berner Zeitschri... 
 subject     | null                 
 title       | Zur Einleitung : ... 
 type        | [Text, Journal Ar... 
only showing top 1 row



In [12]:
# Observe OCR failure in 'publisher'
df.groupBy("publisher").count().orderBy('count', ascending=False).show(truncate=False)

+------------------------------------+-----+
|publisher                           |count|
+------------------------------------+-----+
|Paul Haupt Bern                     |462  |
|Paul Haupt AG Bern                  |173  |
|Historischer Verein des Kantons Bern|84   |
|Rub Media AG                        |79   |
|Paul Haupt AG                       |59   |
|Rub Graf-Lehmann AG                 |46   |
|Paut Haupt Bern                     |13   |
+------------------------------------+-----+



In [13]:
# Check for empty fields
possibly_empty = ['subject', 'coverage', 'relation', 'rights', 'description', 'relation', 'contributor']
for field in possibly_empty:
    print(df.where(df[field] != "null").count())

0
0
0
0
0
0
0


In [14]:
df.select("source").show(5, truncate=False)

+----------------------------------------------------------------------------------------+
|source                                                                                  |
+----------------------------------------------------------------------------------------+
|[Berner Zeitschrift für Geschichte, 2496177-2, 1663-7941, 76, 2014, 3,, 3]              |
|[Berner Zeitschrift für Geschichte, 2496177-2, 1663-7941, 79, 2017, 2,, 57]             |
|[Berner Zeitschrift für Geschichte und Heimatkunde, 280461-x, 0005-9420, 23, 1961,,, 1] |
|[Berner Zeitschrift für Geschichte, 2496177-2, 1663-7941, 77, 2015, 4,, 46]             |
|[Berner Zeitschrift für Geschichte und Heimatkunde, 280461-x, 0005-9420, 5, 1943,,, 161]|
+----------------------------------------------------------------------------------------+
only showing top 5 rows



In [15]:
df.groupBy([col("source")[0]]).count().orderBy('count', ascending=False).show(truncate=False)

+-------------------------------------------------+-----+
|source[0]                                        |count|
+-------------------------------------------------+-----+
|Berner Zeitschrift für Geschichte und Heimatkunde|735  |
|Berner Zeitschrift für Geschichte                |181  |
+-------------------------------------------------+-----+



In [16]:
# Observe duplicate information in 'format'
df.select(col("format")).distinct().show(truncate=False)

+---------------------------------------+
|format                                 |
+---------------------------------------+
|[text/html, application/pdf, text/html]|
+---------------------------------------+



## Cleaning and formatting the data

In [28]:
# Read out 'source' field into several ones
df_journal = df.withColumn('journal',df['source'][0].cast('String'))
df_volume = df_journal.withColumn('volume', df['source'][3].cast('Integer'))
df_issue = df_volume.withColumn('journal', df['source'][5].cast('Integer'))
df_startpage = df_issue.withColumn('start_page', df['source'][7].cast('Integer'))
df_issn = df_startpage.withColumn('issn', df['source'][2].cast('String'))
df_zdb_id = df_issn.withColumn('issn', df['source'][1].cast('String')) \
            .drop(df_issn['source'])

# Add field for source collection
df_coll = df_zdb_id.withColumn("source_collection", lit('E-Periodica'))

# Clean a OCR failure in field 'publisher'
df_publisher = df_coll.withColumn('publisher_new', \
                        when(col('publisher') == 'Paut Haupt Bern', \
                             lit('Paul Haupt Bern')) \
                             .otherwise(df_issn['publisher'].cast('String'))) \
                                .drop(df_issn['publisher'])
df_publisher = df_publisher.withColumnRenamed('publisher_new', 'publisher')

# Make a new format field which cleans duplicate information
# Still some bug in here...
df_form_1 = df_publisher.withColumn('format_new_1', df_publisher['format'][0].cast('String'))

df_form_2 = df_form_1.withColumn('format_new_2', df_form_1['format'][1].cast('String'))

df_form_3 = df_form_2.withColumn('format_new', concat(df_form_2.format_new_1, lit(','), df_form_2.format_new_2)) \
                .drop(df_form_2['format'])
df_form_4 = df_form_3.withColumn('format', split(col('format_new'), ',').cast("array<int>")) \
                .drop(df_form_3['format_new_1']).drop(df_form_3['format_new_2']).drop(df_form_3['format_new'])

# Rename 'type' to prevent confusion in Elasticsearch use
df_cleaned = df_form_4.withColumnRenamed('type', 'document_type')

df_cleaned.persist().printSchema()

root
 |-- contributor: string (nullable = true)
 |-- coverage: string (nullable = true)
 |-- creator: string (nullable = true)
 |-- date: string (nullable = true)
 |-- description: string (nullable = true)
 |-- fulltext: string (nullable = true)
 |-- id_intern: string (nullable = true)
 |-- identifier: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- language: string (nullable = true)
 |-- relation: string (nullable = true)
 |-- rights: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- title: string (nullable = true)
 |-- document_type: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- journal: integer (nullable = true)
 |-- volume: integer (nullable = true)
 |-- start_page: integer (nullable = true)
 |-- issn: string (nullable = true)
 |-- source_collection: string (nullable = false)
 |-- publisher: string (nullable = true)
 |-- format: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [29]:
df_cleaned.show(1, vertical=True)  # show the first row

-RECORD 0---------------------------------
 contributor       | null                 
 coverage          | null                 
 creator           | ["Boser, Lukas","... 
 date              | 2014                 
 description       | null                 
 fulltext          | Zur Einleitung
Er... 
 id_intern         | zgh-002:2014:76::406 
 identifier        | [https://www.e-pe... 
 language          | null                 
 relation          | null                 
 rights            | null                 
 subject           | null                 
 title             | Zur Einleitung : ... 
 document_type     | [Text, Journal Ar... 
 journal           | 3                    
 volume            | 76                   
 start_page        | 3                    
 issn              | 2496177-2            
 source_collection | E-Periodica          
 publisher         | Rub Media AG         
 format            | [,]                  
only showing top 1 row



## Text processing

### Detecting languages

In [30]:
!pip install langdetect
from langdetect import detect
from langdetect import detect_langs

In [31]:
# Test 'detect_langs' gives probability of language(s)
title = "Das Wappen auf dem Lande."
print(detect(title))
print(detect_langs(title))

de
[de:0.9999973049843143]


In [32]:
# Define UDF for detecting languages

def detect_language(text):
    lang = detect(text)
    if lang == 'de':
        language = 'German'
    elif lang == 'fr':
        language = 'French'
    elif lang == 'en':
        language = 'English'
    elif lang == 'la':
        language = 'Latin'
    else: language = 'Other'
    return language

udf_detect_language = udf(detect_language, StringType())

In [33]:
# Apply UDF for detecting languages

df_lang = df_cleaned.drop('language')
df_language = df_lang.withColumn("language", udf_detect_language(col("title")))

In [34]:
df_language.groupBy("language").count().orderBy('count', ascending=False).show(truncate=False)

+--------+-----+
|language|count|
+--------+-----+
|German  |872  |
|French  |21   |
|Other   |20   |
|English |3    |
+--------+-----+



In [35]:
# problem cases with language detection
others = df_language.filter(df_language.language == "Other")
others.select('title').show(30, truncate=False)

+--------------------------------------------+
|title                                       |
+--------------------------------------------+
|Bättlerjegi                                 |
|Fundstück : Bümpliz-Casablanca              |
|Titelvignette                               |
|Niclaus Manuel D                            |
|Emmentaler Eheverträge                      |
|Miszellen                                   |
|Der Fall Bartocci                           |
|David Müslin als Pädagoge                   |
|Vier berner Historiker                      |
|Chroniken von Interlaken 1850-1854          |
|Ingenieur Gustave Bridel (1827-1884)        |
|Vom Bärner Wappe                            |
|Chronik von Interlaken 1850-1854            |
|Die grosse Glocke von Sumiswald             |
|Stift 1745-1750                             |
|Der Merkantilismus im Oberaargau            |
|Neuenegg Anno 1798                          |
|La fête d'interlaken                        |
|KKK [kantona

### Named entity recognition

In [36]:
# Install SpaCy
!pip install -U spacy==3.1       # to force the latest version
import spacy

In [37]:
# Download spaCy large German language model, 571.2 MB
!python -m spacy download de_core_news_lg  
import de_core_news_lg

     |████████████████████████████████| 571.2 MB 9.6 kB/s  eta 0:00:01     |██████▍                         | 113.6 MB 40.4 MB/s eta 0:00:12     |█████████████                   | 231.8 MB 1.0 MB/s eta 0:05:35     |█████████████▊                  | 244.0 MB 1.0 MB/s eta 0:05:23     |██████████████████▎             | 325.6 MB 603 kB/s eta 0:06:48     |███████████████████▌            | 348.4 MB 59.8 MB/s eta 0:00:04     |███████████████████████████▌    | 491.2 MB 82.7 MB/s eta 0:00:01     |████████████████████████████▊   | 513.7 MB 82.7 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_lg')


In [38]:
# Load the language model
nlp = de_core_news_lg.load()              # alternative if problems occur: nlp = spacy.load('de_core_news_lg')     
print("SpaCy language model implemented successfully")

SpaCy language model implemented successfully


In [39]:
spacy.info()

{'spacy_version': '3.1.0',
 'location': '/opt/conda/lib/python3.8/site-packages/spacy',
 'platform': 'Linux-5.4.0-1018-aws-x86_64-with-glibc2.10',
 'python_version': '3.8.6',
 'pipelines': {'de_core_news_lg': '3.1.0'}}

In [40]:
# Define UDF for cleaning the raw text (remove special chars and words < 3) with NLTK

# Install NLTK
!pip install nltk
import nltk  
from nltk import word_tokenize   

def preprocess_nltk(text):
    wordlist = nltk.word_tokenize(str(text), language='german')
    # punctuation: all special characters, but not sentence endings
    punctuation = [',', ';', ':', '(', ')', '[', ']', '{', '}', '\"', '\'','\'\'', '\`', '\`\`', \
                   '\-', '«', '»', '£', '\^', '~', '*', '®', '•', '■', '♦', '§']
    wordlist_stripped = [w for w in wordlist if w not in punctuation]
    wordlist_stripped = [w for w in wordlist if len(w) > 2]
    wordlist = ' '.join(wordlist_stripped)
    return wordlist

udf_nltk_tokenize = udf(preprocess_nltk, StringType())

In [41]:
# Define UDF for applying large Spacy german model on fulltext to recognize named entites of localities and persons

nltk.download('punkt')                # "punkt" = standard classifier for sentence segmentation 
from nltk import sent_tokenize
    
def spacy_ner_loc(text):
    sents = nltk.sent_tokenize(text)
    ner_loc = []
    for s in sents:
        doc = nlp(s)
        ner_loc.append([ent.lemma_ for ent in doc.ents if ent.label_ == 'LOC'])
    return ner_loc

def spacy_ner_per(text):
    sents = nltk.sent_tokenize(text)
    ner_loc = []
    for s in sents:
        doc = nlp(s)
        ner_loc.append([ent.lemma_ for ent in doc.ents if ent.label_ == 'PER'])
    return ner_loc

udf_spacy_ner_loc = udf(spacy_ner_loc, StringType())
udf_spacy_ner_per = udf(spacy_ner_per, StringType())

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [45]:
# Apply text cleaning and NER

df_nltk = df_language.withColumn("nltk", udf_nltk_tokenize(col("fulltext")))

# Extract LOC NER with SpaCy
df_spacy = df_nltk.withColumn("locality", udf_spacy_ner_loc(col("nltk"))).drop("nltk")

# tried -> not sufficient
#df_spacy = df_spacy_loc.withColumn("person", udf_spacy_ner_per(col("nltk"))).drop("nltk")

df_final = df_spacy.persist()

In [295]:
# Not yet in use
'''
def clean_locality(text):
    return re.sub(r'\[+|\]+', '', text)

udf_clean_locality = udf(clean_locality, StringType())

# Clean the 'locality' from duplicate '[' and ']'
df_loc_clean = df_spacy.withColumn("locality_clean", udf_clean_locality(col("locality"))) \
                                   .drop(df_spacy['locality'])
df_final = df_loc_clean.withColumn("locality", concat(lit('['), 'locality_clean', \
                                         lit(']'))).drop(df_loc_clean['locality_clean'])
                                   
df_final.persist()
'''

DataFrame[contributor: string, coverage: string, creator: string, date: string, description: string, fulltext: string, id_intern: string, identifier: array<string>, relation: string, rights: string, subject: string, title: string, document_type: array<string>, journal: int, volume: int, start_page: int, issn: string, source_collection: string, publisher: string, format: string, language: string, locality: string]

In [ ]:
df_final.show(1, vertical=True)

In [277]:
df_final.printSchema()

root
 |-- contributor: string (nullable = true)
 |-- coverage: string (nullable = true)
 |-- creator: string (nullable = true)
 |-- description: string (nullable = true)
 |-- fulltext: string (nullable = true)
 |-- id_intern: string (nullable = true)
 |-- identifier: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- relation: string (nullable = true)
 |-- rights: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- title: string (nullable = true)
 |-- document_type: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- year: integer (nullable = true)
 |-- journal: integer (nullable = true)
 |-- volume: integer (nullable = true)
 |-- start_page: integer (nullable = true)
 |-- issn: string (nullable = true)
 |-- source_collection: string (nullable = false)
 |-- publisher: string (nullable = true)
 |-- format_new: string (nullable = true)
 |-- language: string (nullable = true)
 |-- locality: string (nullable = true)



# Ingest to Elasticsearch

In [224]:
# Install the Elasticsearch APIs
!pip install elasticsearch
!pip install elasticsearch-dsl
import elasticsearch
import elasticsearch_dsl

In [5]:
# Configure the basic API with a Elasticsearch client
from elasticsearch import Elasticsearch
es = Elasticsearch("elasticsearch-1")
es.info()

{'name': 'elasticsearch-1',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'ESiZyajNQz-wfYyashdn4Q',
 'version': {'number': '7.10.1',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '1c34507e66d7db1211f66f3513706fdf548736aa',
  'build_date': '2020-12-05T01:00:33.671820Z',
  'build_snapshot': False,
  'lucene_version': '8.7.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [ ]:
# Create explicit mapping for the index
# See https://www.elastic.co/guide/en/elasticsearch/reference/7.x/explicit-mapping.html
# and https://www.elastic.co/guide/en/elasticsearch/reference/7.x/mapping-params.html

In [210]:
%%bash
# Test standard analyzer
curl -X POST "http://172.26.6.171:9200/_analyze?pretty" -H 'Content-Type: application/json' -d'
{
  "analyzer": "standard",
  "text": "D\u00fcrfte von Anfang. Ein anderer Hinweis als\nselbstverst\u00e4ndlich."
}
'

{
  "tokens" : [
    {
      "token" : "dürfte",
      "start_offset" : 0,
      "end_offset" : 6,
      "type" : "<ALPHANUM>",
      "position" : 0
    },
    {
      "token" : "von",
      "start_offset" : 7,
      "end_offset" : 10,
      "type" : "<ALPHANUM>",
      "position" : 1
    },
    {
      "token" : "anfang",
      "start_offset" : 11,
      "end_offset" : 17,
      "type" : "<ALPHANUM>",
      "position" : 2
    },
    {
      "token" : "ein",
      "start_offset" : 19,
      "end_offset" : 22,
      "type" : "<ALPHANUM>",
      "position" : 3
    },
    {
      "token" : "anderer",
      "start_offset" : 23,
      "end_offset" : 30,
      "type" : "<ALPHANUM>",
      "position" : 4
    },
    {
      "token" : "hinweis",
      "start_offset" : 31,
      "end_offset" : 38,
      "type" : "<ALPHANUM>",
      "position" : 5
    },
    {
      "token" : "als",
      "start_offset" : 39,
      "end_offset" : 42,
      "type" : "<ALPHANUM>",
      "position" : 6
    },
    {


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1271  100  1154  100   117  1126k   114k --:--:-- --:--:-- --:--:-- 1241k


In [242]:
%%bash
# Build the index via settings & mapping
# Use private IP

curl -X PUT "http://172.26.6.171:9200/index_6" -H 'Content-Type: application/json' -d'
{
"settings": {
    "analysis": {
        "analyzer": {
            "my_german": {
                "type": "standard",
                "stopwords": "_german_" }
      },
        "normalizer": {
            "my_normalizer": {
                "type": "custom",
                "char_filter": [],
                "filter": ["lowercase", "asciifolding"]
            }
          }
        }
      },
    "mappings" : {
        "properties" : {
            "creator" : {
                "type" : "text",
                "index_prefixes": { },
                "dynamic": "false",
                "fields" : {
                    "keyword" : {
                        "type" : "keyword",
                        "normalizer": "my_normalizer" }
          }
        },
            "year" : {
                "type": "date",
                "format": "yyyy",
                "dynamic": "false",
                "fields" : {
                    "keyword" : {
                        "type" : "keyword" }
          }
        },
            "volume" : {
                "type":   "integer",
                "dynamic": "false",
                "fields" : {
                    "keyword" : {
                        "type" : "keyword" }
          }
        },
            "issue" : {
                "type":   "integer",
                "dynamic": "false",
                "fields" : {
                    "keyword" : {
                        "type" : "keyword" }
          }
        },
            "start_page" : {
                "type":   "integer",
                "dynamic": "false",
                "fields" : {
                    "keyword" : {
                        "type" : "keyword" }
          }
        },
            "issn" : {
                "type":   "text",
                "dynamic": "false",
                "fields" : {
                    "keyword" : {
                        "type" : "keyword" }
          }
        },
            "zdb_id" : {
                "type": "text",
                "dynamic": "false",
                "fields" : {
                    "keyword" : {
                        "type" : "keyword" }
          }
        },
            "format" : {
                "type" : "text",
                "dynamic": "false",
                "index_prefixes": { }, 
                "fields" : {
                    "keyword" : {
                        "type" : "keyword",
                        "normalizer": "my_normalizer" }
          }
        },
            "id_intern" : {
                "type" : "text",
                "dynamic": "false",
                "index_prefixes": { },
                "fields" : {
                    "keyword" : {
                        "type" : "keyword",
                        "normalizer": "my_normalizer" }
          }
        },
            "identifier" : {
                "type" : "text",
                "dynamic": "false",
                "index_prefixes": { },
                "fields" : {
                    "keyword" : {
                        "type" : "keyword" }
          }
        },
            "locality" : {
                "type" : "text",
                "dynamic": "false",
                "index_prefixes": { }, 
                "fields" : {
                    "keyword" : {
                        "type" : "keyword",
                        "normalizer": "my_normalizer" },
                    "german": {
                        "type": "text",
                        "analyzer": "my_german" }
          }
        },                        
            "publisher" : {
                "type" : "text",
                "dynamic": "false",
                "index_prefixes": { },
                "fields" : {
                    "keyword" : {
                        "type" : "keyword",
                        "normalizer": "my_normalizer" }
          }
        },
            "title" : {
                "type" : "text",
                "dynamic": "false",
                "index_prefixes": { }, 
                "fields" : {
                    "keyword" : {
                        "type" : "keyword",
                        "normalizer": "my_normalizer" },
                    "german": {
                        "type": "text",
                        "analyzer": "my_german" }
          }
        },    
            "fulltext" : {
                "type" : "text",
                "dynamic": "false",
                "index_prefixes": { }, 
                "fields" : {
                    "keyword" : {
                        "type" : "keyword",
                        "normalizer": "my_normalizer" },
                    "german": {
                        "type": "text",
                        "analyzer": "my_german" }
          }
        },    
            "document_type" : {
                "type" : "text",
                "dynamic": "false",
                "index_prefixes": { },
                "fields" : {
                    "keyword" : {
                        "type" : "keyword",
                        "normalizer": "my_normalizer" }
          }
        },
            "journal" : {
                "type" : "text",
                "dynamic": "false",
                "index_prefixes": { },
                "fields" : {
                    "keyword" : {
                        "type" : "keyword",
                        "normalizer": "my_normalizer" }
          }
        },
            "source_collection" : {
                "type" : "text",
                "dynamic": "false",
                "index_prefixes": { },
                "fields" : {
                    "keyword" : {
                        "type" : "keyword",
                        "normalizer": "my_normalizer" }
          }
        },
            "language" : {
                "type" : "text",
                "dynamic": "false",
                "index_prefixes": { },
                "fields" : {
                    "keyword" : {
                        "type" : "keyword",
                        "normalizer": "my_normalizer" }
          }
        }
      }
    }
}
'

{"error":{"root_cause":[{"type":"mapper_parsing_exception","reason":"unknown parameter [dynamic] on mapper [identifier] of type [text]"}],"type":"mapper_parsing_exception","reason":"Failed to parse mapping [_doc]: unknown parameter [dynamic] on mapper [identifier] of type [text]","caused_by":{"type":"mapper_parsing_exception","reason":"unknown parameter [dynamic] on mapper [identifier] of type [text]"}},"status":400}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6482  100   420  100  6062  70000   986k --:--:-- --:--:-- --:--:-- 1055k


In [240]:
%%bash
# Build the index via settings & mapping
# Use private IP

curl -X PUT "http://172.26.6.171:9200/index_6" -H 'Content-Type: application/json' -d'
{
"index_6": {
    "settings": {
        "analysis": {
            "analyzer": {
                "my_german": {
                    "type": "standard",
                    "stopwords": "_german_" }
          },
            "normalizer": {
                "my_normalizer": {
                    "type": "custom",
                    "char_filter": [],
                    "filter": ["lowercase", "asciifolding"]
                }
              }
            }
          },
        "mappings" : {
            "eperiodica": {
                "dynamic": false,
                "properties" : {
                    "creator" : {
                        "type" : "text",
                        "index_prefixes": { },
                        "fields" : {
                            "keyword" : {
                                "type" : "keyword",
                                "normalizer": "my_normalizer" }
                  }
                },
                    "year" : {
                        "type": "date",
                        "format": "yyyy",
                        "fields" : {
                            "keyword" : {
                                "type" : "keyword" }
                  }
                },
                    "volume" : {
                        "type":   "integer",
                        "fields" : {
                            "keyword" : {
                                "type" : "keyword" }
                  }
                },
                    "issue" : {
                        "type":   "integer",
                        "fields" : {
                            "keyword" : {
                                "type" : "keyword" }
                  }
                },
                    "start_page" : {
                        "type":   "integer",
                        "fields" : {
                            "keyword" : {
                                "type" : "keyword" }
                  }
                },
                    "issn" : {
                        "type":   "text",
                        "fields" : {
                            "keyword" : {
                                "type" : "keyword" }
                  }
                },
                    "zdb_id" : {
                        "type": "text",
                        "fields" : {
                            "keyword" : {
                                "type" : "keyword" }
                  }
                },
                    "format" : {
                        "type" : "text",
                        "index_prefixes": { }, 
                        "fields" : {
                            "keyword" : {
                                "type" : "keyword",
                                "normalizer": "my_normalizer" }
                  }
                },
                    "id_intern" : {
                        "type" : "text",
                        "index_prefixes": { },
                        "fields" : {
                            "keyword" : {
                                "type" : "keyword",
                                "normalizer": "my_normalizer" }
                  }
                },
                    "identifier" : {
                        "type" : "text",
                        "index_prefixes": { },
                        "fields" : {
                            "keyword" : {
                                "type" : "keyword" }
                  }
                },
                    "locality" : {
                        "type" : "text",
                        "index_prefixes": { }, 
                        "fields" : {
                            "keyword" : {
                                "type" : "keyword",
                                "normalizer": "my_normalizer" },
                            "german": {
                                "type": "text",
                                "analyzer": "my_german" }
                  }
                },                        
                    "publisher" : {
                        "type" : "text",
                        "index_prefixes": { },
                        "fields" : {
                            "keyword" : {
                                "type" : "keyword",
                                "normalizer": "my_normalizer" }
                  }
                },
                    "title" : {
                        "type" : "text",
                        "index_prefixes": { }, 
                        "fields" : {
                            "keyword" : {
                                "type" : "keyword",
                                "normalizer": "my_normalizer" },
                            "german": {
                                "type": "text",
                                "analyzer": "my_german" }
                  }
                },    
                    "fulltext" : {
                        "type" : "text",
                        "index_prefixes": { }, 
                        "fields" : {
                            "keyword" : {
                                "type" : "keyword",
                                "normalizer": "my_normalizer" },
                            "german": {
                                "type": "text",
                                "analyzer": "my_german" }
                  }
                },    
                    "document_type" : {
                        "type" : "text",
                        "index_prefixes": { },
                        "fields" : {
                            "keyword" : {
                                "type" : "keyword",
                                "normalizer": "my_normalizer" }
                  }
                },
                    "journal" : {
                        "type" : "text",
                        "index_prefixes": { },
                        "fields" : {
                            "keyword" : {
                                "type" : "keyword",
                                "normalizer": "my_normalizer" }
                  }
                },
                    "source_collection" : {
                        "type" : "text",
                        "index_prefixes": { },
                        "fields" : {
                            "keyword" : {
                                "type" : "keyword",
                                "normalizer": "my_normalizer" }
                  }
                },
                    "language" : {
                        "type" : "text",
                        "index_prefixes": { },
                        "fields" : {
                            "keyword" : {
                                "type" : "keyword",
                                "normalizer": "my_normalizer" }
                  }
                }
              }
            }
        }
    }
}
'

{"error":"Incorrect HTTP method for uri [/] and method [PUT], allowed: [GET, DELETE, HEAD]","status":405}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6974  100   105  100  6869   102k  6708k --:--:-- --:--:-- --:--:-- 6810k


In [281]:
# Ingest to Elasticsearch as JSON
# https://www.elastic.co/guide/en/elasticsearch/hadoop/7.x/spark.html
# See settings https://www.elastic.co/guide/en/elasticsearch/hadoop/7.x/configuration.html
from pyspark.sql import SQLContext

esconf={}
esconf["es.resource"] = "index_5/eperiodica"    # format: index/type
esconf["es.mapping.id"] = "id_intern"         # if 'id_intern' should be used as global ID in ES -> updating!
esconf["es.nodes"] = "elasticsearch-1"
# OR
#esconf["es.nodes"] = IP                        # default: 'localhost'
#esconf["es.port"] = "9200" 

#esconf["es.mapping.include"] = None             # fields to include with ingest
#esconf["es.mapping.exclude"] = None             # fields to exclude with ingest

#esconf["es.read.field.as.array.exclude"] (default empty)   # Fields/properties that should NOT be considered as arrays/lists
#esconf["es.read.field.as.array.include"] = nested.bar:3    # maps nested.bar as a 3-level/dimensional arra

esconf["es.write.operation"] = "upsert"
#index (default): new data is added while existing data (based on its id) is replaced (reindexed). 
#create: adds new data - if the data already exists (based on its id), an exception is thrown. 
#update: updates existing data (based on its id). If no data is found, an exception is thrown. 
#upsert: known as merge or insert if the data does not exist, updates if the data exists (based on its id). 

df_final.write.format("org.elasticsearch.spark.sql") \
                    .options(**esconf).save()    # .mode('append') for adding fields

Py4JJavaError: An error occurred while calling o1759.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 186.0 failed 1 times, most recent failure: Lost task 0.0 in stage 186.0 (TID 2933, jupyter, executor driver): org.elasticsearch.hadoop.EsHadoopException: Could not write all entries for bulk operation [30/30]. Error sample (first [5] error messages):
	org.elasticsearch.hadoop.rest.EsHadoopRemoteException: illegal_argument_exception: Mapper for [source_collection.keyword] conflicts with existing mapper:
	Cannot update parameter [normalizer] from [my_normalizer] to [default]
	{"update":{"_id":"zgh-002:2014:76::406"}}
{"doc_as_upsert":true,"doc":{"creator":"[\"Boser, Lukas\",\"Hofmann, Michèle\"]","fulltext":"Zur Einleitung\nErnährung macht Schule\n\n-\nseit 200 Jahren\n\nLw/ca.s Böser und Mz'c/zèZe Bofmann\n\nDie Frage der «richtigen» oder gesunden Ernährung hat in den vergangenen\nJahren in der Schweizer Volksschule stark an Bedeutung gewonnen. 

In [282]:
# Easy ingest to Elasticsearch as JSON
from pyspark.sql import SQLContext

df_final.write.format("org.elasticsearch.spark.sql") \
    .option("es.resource", "index/eperiodica") \
    .option("es.mapping.id", "id_intern") \
    .option("es.write.operation", "index") \
    .option("es.nodes", "elasticsearch-1").save()

# Query Elasticsearch

In [6]:
es.cluster.health(wait_for_status='yellow', request_timeout=1)   # 6 shards are default

{'cluster_name': 'docker-cluster',
 'status': 'yellow',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'active_primary_shards': 7,
 'active_shards': 7,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 1,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 87.5}

In [11]:
# Whole mapping of index
! curl -X GET "http://172.26.6.171:9200/index_1/_mapping?pretty" > mapping_index_1.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2324  100  2324    0     0  1134k      0 --:--:-- --:--:-- --:--:-- 1134k


In [10]:
# Search on field of index + type via REST API
# Use private IP

#! curl -GET "http://172.26.6.171:9200/index_1/eperiodica/_search?q=title:aare&pretty"

In [39]:
# Simple search via DSL API
from elasticsearch_dsl import Search

s = Search().using(es).query("match", publisher="verein")    # fulltext="bern"
response = s.execute()
print(response, "\n")
for hit in s:
    print(hit.title, '\n\n', hit.publisher)

<Response: [<Hit(index_1/pB-tT3wBsgv0SeAE1D_2): {'creator': 'Singeisen, Raphael', 'date': '2018', 'format': ...}>]> 

Aus Wald wird Ackerland : Kriegsrodungen im Kanton Bern 1941-1946 : ein umstrittenes Kapitel der Anbauschlacht 

 Historischer Verein des Kantons Bern


In [43]:
s = Search()\
        .using(es)\
        .query("match", fulltext="frau")\
        .extra(track_total_hits=True)

response = s.execute()
print('Total %d hits found.' % response.hits.total.value)

h = response.hits[0]
print('/%s/%s/%s returned with score %f' % (
    h.meta.index, h.meta.doc_type, h.meta.id, h.meta.score))

Total 213 hits found.
/index_1/eperiodica/hB-tT3wBsgv0SeAE1D-A returned with score 2.094948


In [46]:
print(response.success())
print(response.took)
print(response.hits.total.relation)
print(response.hits.total.value)

True
6
eq
213


In [45]:
for hit in s[0]:
    print(hit.meta)
    print(hit.meta.score)
    print(hit.title)
    print(hit.creator)
    print(hit.id_intern)
    print(hit.localities)

{'index': 'index_1', 'id': 'hB-tT3wBsgv0SeAE1D-A', 'score': ...}
2.0949483
Fremder Besuch im Oberland um 1810 : kleine Hörszene von Christian Lerch, gesendet von Radio Bern im Mai 1940
Lerch, Christian
zgh-001:1944:6::275
[[Pfarrhaus Lauterbrunnen, Ahlfeld, Bern, Kuli, Schweizerisch, Ahlf, you know, Europa, Schweiz, Schweiz, Ahlf, Seje cha nid, Lippe, Überhoupt, Staubbach, Staubbach, Berg, Ahlf, begei¬, Ahlf, Scotland, Ahlf, ge¬, Ahlf Huuh, Ahlf, Bergabgehen, Ahlf, Ahlf, Oberland, Hörszene]]


In [54]:
s = Search().using(es)\
        .query("multi_match", query='winterthur', fields=['title', 'localities'])
#s = s.extra(explain=True)

response = s.execute()
for hit in s:
    print(hit.title)
    print(hit.localities)
    print('---')

Die Zwangsanleihen Massenas : ein Bild aus dunkler Zeit
[[Frankreich, Frankreich, Schweiz, Frankreich, Glarus, Gotthard, Zürich, Österreicher, Stadt, Russe, Österreicher, Waldstätten, Linth, Paris, Stadt Zürich, Zürich, Frankreich, Massena, Stadt Zürich, St. Galle, Zürich, Bürgerschaft, Zürich, Stadt, Zürich, Zürich, Stadt Zürich, Basel, Stadt Basel, Stadt Basel, Stadt Winterthur, Winterthur, Basel, Frankreich, Zürich, Winterthur, Basel, Bern, Schweiz, Frankreich, Binningen, Basler, Stadt, Stadt Basel, Helvetiens, Basel, Stadt Basel, Basel]]
---
Der Hafner Heinrich Hess uns sein Hand- und Hausbuch : ein Beitrag zur Geschichte der stadtbernischen Hafnerei am Ende des 17. Jahrhunderts. Teil 2
[[Biel, Basel, Johannesen, Zürich, Zürich, Züricher, Zürich, Zürich, Zollikon Zürich, Winterthur, Schiben, Reutlinger, Jegenstorf, Winterthur, gsin, gsin, Brugg, Jegenstorf, Schneckenhäfeli, bruni, gsin, Weinachten, Burgdorf, Burdlofer Heiri, Lichtmeß, Lichtmeß, Giben, Burgdorf, Burgdorf, Badhaus, B

In [48]:
## Search Elasticsearch API via "scan"

In [34]:
# Simple search via Elasticsearch API  and 'helpers' module
from elasticsearch.helpers import scan

search = scan(es,
    query={"query": {"match": {"title": "biel"}}},
    index="index_1"         
)
[[i['_source']['title'], i['_source']['localities']] for i in search]

[['Die Apotheker der Stadt Biel : Vortrag gehalten an der Jahresversammlung des Kant. Apothekervereins in St. Immer, 10. November 1957',
  "[[Schweiz, Basel, Apothekenwesen, Biel, Schweiz, Biel, Apothekerwesens, Stadtarchivs, Säßhaus, Graubünden, Hintersäßen, Biel, Bieler, Bern, Bieler, Biel, nisgeld, Wyttenbach, Bern, Stadt Bern, Corallen-, Nördlingen, Biel, Biel, Berner, Biel, Biel, Land, Bieler, Venneramt, Armengutes, Darmstadt, Ritter'sche Apotheke, Burgplatz, Stadt, Chorgerichtshandels, Provisorenstelle, Eckhaus Ring, Franzose, Biel, Haus der obern Apotheke, Biel, Bern, Windsheim, Biel, Bern, Bartolomäus Knecht, Haus der Bäcker Müller der Schmiedengasse, Bacharach, Nidau, Biel, Bern, Bern, Knecht'schen Apotheke Bern, Leipzig, Provisorenstellung, Bern, Haus der Schmiedengasse, Willstädt, Heidenheim, Biel, Basel, Brotschal, Lausanne, Württemberger, Glarus, Bern, Brennkirschen, England, Stadt, Frankreich, Biel, Biel, Basel, Neuenburg, Biel, Zürich, Jurahöhen, Orbe, Lausanne, Genf, Ju